In [1]:
import configparser
import praw
from confluent_kafka import Producer
import socket
import json

In [2]:
# Read passwords and secrets from config file
config_parser = configparser.ConfigParser()
config_parser.read("src/configuration/config.cfg")

['src/configuration/config.cfg']

In [3]:
# Set parameters
reddit_client_id = config_parser["praw"]["client_id"]
reddit_client_secret = config_parser["praw"]["client_secret"]
reddit_password = config_parser["praw"]["password"]
reddit_username = config_parser["praw"]["username"]
reddit_agent = config_parser["praw"]["user_agent"] + reddit_username
sub_reddit = config_parser["praw"]["subreddit"]

In [4]:
# Establish connection to reddit
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_client_secret,
    password=reddit_password,
    user_agent=reddit_agent,
    username=reddit_username,
)

In [5]:
conf = {'bootstrap.servers': "broker:29092",
        'client.id': socket.gethostname()}

producer = Producer(conf)

In [6]:
subreddit = reddit.subreddit(sub_reddit)

for comment in subreddit.stream.comments():
    print(vars(comment))
    #str_comment = str(vars(comment))
    #json_comment = json.dumps(str_comment)
    #data = json.loads(json_comment)
    #producer.produce("comments", key=comment.id, value=data)

{'_replies': [], '_submission': None, '_reddit': <praw.reddit.Reddit object at 0x7fb7a8807070>, 'total_awards_received': 0, 'approved_at_utc': None, 'comment_type': None, 'awarders': [], 'mod_reason_by': None, 'banned_by': None, 'author_flair_type': 'text', 'removal_reason': None, 'link_id': 't3_nova45', 'author_flair_template_id': None, 'likes': None, 'user_reports': [], 'saved': False, 'id': 'h06fbp7', 'banned_at_utc': None, 'mod_reason_title': None, 'gilded': 0, 'archived': False, 'no_follow': True, 'author': Redditor(name='pl3xatr0n'), 'num_comments': 9, 'edited': False, 'can_mod_post': False, 'created_utc': 1622543104.0, 'send_replies': True, 'parent_id': 't1_h06e9sl', 'score': 1, 'author_fullname': 't2_35dmevoq', 'over_18': False, 'treatment_tags': [], 'approved_by': None, 'mod_note': None, 'all_awardings': [], 'subreddit_id': 't5_2qh13', 'body': 'Yes, agreed.\n\nBut I’m pretty sure talc is a stone (?) and it has fibres that basically mimic or appear very close in size to asbesto

KeyboardInterrupt: 